## Find Cells of Interest
Another way to look for data is to search for cells with interesting tuning properties.  We have pre-computed a set of cell metrics.  Take a look at <a href='http://alleninstitute.github.io/AllenSDK/brain_observatory_metrics.html'>this page</a> to see the full list.

You can use these properties to filter the full set of cells down to the set your are interested in.

In [1]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import pprint
boc = BrainObservatoryCache(manifest_file='boc/manifest.json')
import pandas as pd

In [2]:
visp_ecs = boc.get_experiment_containers(targeted_structures=['VISp'])

# Download cells for a set of experiments and convert to DataFrame
cells = boc.get_cell_specimens()
cells = pd.DataFrame.from_records(cells)
print("total cells: %d" % len(cells))

# find direction selective cells in VISp
visp_ec_ids = [ ec['id'] for ec in visp_ecs ]
visp_cells = cells[cells['experiment_container_id'].isin(visp_ec_ids)]
print("VISp cells: %d" % len(visp_cells))

# significant response to drifting gratings stimulus
sig_cells = visp_cells[visp_cells['p_dg'] < 0.05]
print("cells with sig. response to drifting gratings: %d" % len(sig_cells))

# direction selective cells
dsi_cells = sig_cells[(sig_cells['g_dsi_dg'] > 0.9)]
print("direction-selective cells: %d" % len(dsi_cells))

total cells: 37091
VISp cells: 12127
cells with sig. response to drifting gratings: 6610
direction-selective cells: 686


## Find Experiments for Cells
Once you have found a set of cells you would like to investigate, you can use the `BrainObservatoryCache` to find the experiments for those cells that contain the relevant stimulus.

In [3]:
import allensdk.brain_observatory.stimulus_info as stim_info

# find experiment containers for those cells
dsi_ec_ids = dsi_cells['experiment_container_id'].unique()
print("total dsi experiment containers: %d" % len(dsi_ec_ids))

# Download the ophys experiments containing the drifting gratings stimulus for VISp experiment containers
dsi_exps = boc.get_ophys_experiments(experiment_container_ids=dsi_ec_ids, stimuli=[stim_info.DRIFTING_GRATINGS])
print("VISp drifting gratings ophys experiments: %d" % len(dsi_exps))

print("Example ophys experiment:")
pprint.pprint(dsi_exps[11])

total dsi experiment containers: 45
VISp drifting gratings ophys experiments: 45
Example ophys experiment:
{'acquisition_age_days': 157,
 'cre_line': 'Nr5a1-Cre',
 'donor_name': '279430',
 'experiment_container_id': 569811199,
 'fail_eye_tracking': True,
 'id': 569811201,
 'imaging_depth': 300,
 'reporter_line': 'Ai93(TITL-GCaMP6f)',
 'session_type': 'three_session_A',
 'specimen_name': 'Nr5a1-Cre;Camk2a-tTA;Ai93-279430',
 'targeted_structure': 'VISp'}


## Download Experiment Data for a Cell
Once you have some experiments, you can download the NWB files that contain the fluorescence traces for segmented cells in those experiments.

In [4]:
ecid = dsi_exps[11]['experiment_container_id']

# pick a direction-selective cell and find its NWB file
dsi_cell = dsi_cells[dsi_cells['experiment_container_id']==ecid].iloc[0]

# figure out which ophys experiment has the drifting gratings stimulus for that cell
cell_exp = boc.get_ophys_experiments(cell_specimen_ids=[dsi_cell['cell_specimen_id']],
                                     stimuli=[stim_info.DRIFTING_GRATINGS])[0]

data_set = boc.get_ophys_experiment_data(cell_exp['id'])

print("Metadata from NWB file:")
pprint.pprint(data_set.get_metadata())

print("stimuli available in this file:")
print(data_set.list_stimuli())

Metadata from NWB file:
{'age_days': 158,
 'cre_line': 'Nr5a1-Cre/wt',
 'device': 'Nikon A1R-MP multiphoton microscope',
 'device_name': 'CAM2P.2',
 'excitation_lambda': '910 nanometers',
 'experiment_container_id': 569811199,
 'fov': '400x400 microns (512 x 512 pixels)',
 'genotype': 'Nr5a1-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6f)/Ai93(TITL-GCaMP6f)',
 'imaging_depth_um': 300,
 'indicator': 'GCaMP6f',
 'ophys_experiment_id': 569811201,
 'pipeline_version': '2.0',
 'session_start_time': datetime.datetime(2017, 2, 14, 12, 43, 41),
 'session_type': 'three_session_A',
 'sex': 'male',
 'specimen_name': 'Nr5a1-Cre;Camk2a-tTA;Ai93-279430',
 'targeted_structure': 'VISp'}
stimuli available in this file:
['drifting_gratings', 'natural_movie_one', 'natural_movie_three', 'spontaneous']


In [5]:
dsi_cell

all_stim                                                                   False
area                                                                        VISp
cell_specimen_id                                                       572392558
donor_full_genotype            Nr5a1-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6f)/...
dsi_dg                                                                  0.947851
experiment_container_id                                                569811199
failed_experiment_container                                                False
g_dsi_dg                                                                0.941699
g_osi_dg                                                                0.996283
g_osi_sg                                                                0.634511
image_sel_ns                                                            0.367424
imaging_depth                                                                300
osi_dg                      